In [2]:
!pip install numpy pandas matplotlib seaborn plotly requests tqdm opencv-python pillow wandb -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## Download and install Pytorch

In [3]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://download.pytorch.org/whl/cu113


In [4]:
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/dataset/SimHei.ttf --no-check-certificate


--2024-02-26 16:29:54--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/dataset/SimHei.ttf
Connecting to 172.16.0.13:5848... connected.
Proxy request sent, awaiting response... 200 OK
Length: 10050868 (9.6M) [application/x-font-ttf]
Saving to: ‘SimHei.ttf.2’

SimHei.ttf.2        100%[===================>]   9.58M  16.7MB/s    in 0.6s    

2024-02-26 16:29:55 (16.7 MB/s) - ‘SimHei.ttf.2’ saved [10050868/10050868]



## Create a catalogue

In [5]:
import os

In [6]:
# Store the results file
# os.mkdir('output')

# Store the trained model weights
os.mkdir('checkpoint')


# Store the generated charts
os.mkdir('diagrams')

FileExistsError: [Errno 17] File exists: 'checkpoint'

## Setting matplotlib Chinese and English fonts

In [ ]:
## Font Environment Settings
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.font_manager import FontProperties
 
# global font settings
SimSun = FontProperties(fname='/home/featurize/SimHei.ttf')  # Used to display Chinese labels normally
plt.rcParams['axes.unicode_minus'] = False  # Used to display the negative sign normally
Times_New_Roman = FontProperties(fname='/home/featurize/times.ttf')
 
# mixed font settings
config = {
#     "font.family":'serif',
#     "font.size": 80,
       "mathtext.fontset":'stix',
#     "font.serif": ['SimSun'],
}
rcParams.update(config)
 
#Canvas Settings
fig = plt.figure(num=1, figsize=(9, 6),dpi=180)
ax = plt.axes((0.23,0.23,0.6,0.6))
 
 
# Application of font effects 
ax.set_title('中文宋体 $\mathrm{Times}$ $\mathrm{New}$ $\mathrm{Roman}$ $\mathrm{123}$'\
                                           ,fontproperties=SimSun,fontsize=12)  
 
ax.set_xlabel('测试测试',fontproperties=SimSun,fontsize=12)                      
 
ax.set_ylabel('TestTest',fontproperties=Times_New_Roman,fontsize=12)            
 
 
plt.show()

In [7]:
!sudo snap install tree

snap "tree" is already installed, see 'snap help refresh'


In [8]:
!tree /home/featurize/data -L 2

/home/featurize/data
├── train
│   ├── CherryTomatoes
│   ├── Mangosteen
│   ├── MomordicaCharantia
│   ├── NavelOrange
│   ├── Sandsugaroranges
│   ├── apple
│   ├── banana
│   ├── carrot
│   ├── cherries
│   ├── cucumber
│   ├── durian
│   ├── grape
│   ├── hamimelon
│   ├── kiwi
│   ├── lemon
│   ├── lichee
│   ├── longan
│   ├── mango
│   ├── pear
│   ├── pineapple
│   ├── pitaya
│   ├── pomegranate
│   ├── strawberry
│   ├── tomato
│   └── watermelon
└── val
    ├── Cherrytomatoes
    ├── Mangosteen
    ├── MomordicaCharantia
    ├── NavelOrange
    ├── Sandsugaroranges
    ├── apple
    ├── banana
    ├── carrot
    ├── cherries
    ├── cucumber
    ├── durian
    ├── grape
    ├── hamimelon
    ├── kiwi
    ├── lemon
    ├── lichee
    ├── longan
    ├── mango
    ├── pear
    ├── pineapple
    ├── pitaya
    ├── pomegranate
    ├── strawberry
    ├── tomato
    └── watermelon

52 directories, 0 files


# change


In [9]:
import time
import os
from tqdm import tqdm

import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline


import warnings
warnings.filterwarnings("ignore")


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


In [11]:
from torchvision import transforms

# Training Set Image Preprocessing - RCTN: Scaling, Cropping, Turn Tensor, Normalisation
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])

# Test Set Image Preprocessing - RCTN: Scaling, Cropping, Turn Tensor, Normalisation
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

In [12]:
# Dataset folder path
dataset_dir = '/home/featurize/data'

In [13]:
train_path = os.path.join(dataset_dir, 'train')
test_path = os.path.join(dataset_dir, 'val')
print('Training_set_path', train_path)
print('Testing_set_path', test_path)

Training_set_path /home/featurize/data/train
Testing_set_path /home/featurize/data/val


In [14]:
from torchvision import datasets

# Load training set
train_dataset = datasets.ImageFolder(train_path, train_transform)

# Load Test Set
test_dataset = datasets.ImageFolder(test_path, test_transform)

In [15]:
print('Number of images in the training set', len(train_dataset))
print('Number of categories', len(train_dataset.classes))
print('Name of each category', train_dataset.classes)

Number of images in the training set 3649
Number of categories 25
Name of each category ['CherryTomatoes', 'Mangosteen', 'MomordicaCharantia', 'NavelOrange', 'Sandsugaroranges', 'apple', 'banana', 'carrot', 'cherries', 'cucumber', 'durian', 'grape', 'hamimelon', 'kiwi', 'lemon', 'lichee', 'longan', 'mango', 'pear', 'pineapple', 'pitaya', 'pomegranate', 'strawberry', 'tomato', 'watermelon']


In [16]:
print('Number of test set images', len(test_dataset))
print('Number of categories', len(test_dataset.classes))
print('Name of each category', test_dataset.classes)

Number of test set images 898
Number of categories 25
Name of each category ['Cherrytomatoes', 'Mangosteen', 'MomordicaCharantia', 'NavelOrange', 'Sandsugaroranges', 'apple', 'banana', 'carrot', 'cherries', 'cucumber', 'durian', 'grape', 'hamimelon', 'kiwi', 'lemon', 'lichee', 'longan', 'mango', 'pear', 'pineapple', 'pitaya', 'pomegranate', 'strawberry', 'tomato', 'watermelon']


In [17]:
# Name of each category
class_names = train_dataset.classes
n_class = len(class_names)

In [18]:
class_names

['CherryTomatoes',
 'Mangosteen',
 'MomordicaCharantia',
 'NavelOrange',
 'Sandsugaroranges',
 'apple',
 'banana',
 'carrot',
 'cherries',
 'cucumber',
 'durian',
 'grape',
 'hamimelon',
 'kiwi',
 'lemon',
 'lichee',
 'longan',
 'mango',
 'pear',
 'pineapple',
 'pitaya',
 'pomegranate',
 'strawberry',
 'tomato',
 'watermelon']

In [19]:
# Mapping relationship: category to index number
train_dataset.class_to_idx

{'CherryTomatoes': 0,
 'Mangosteen': 1,
 'MomordicaCharantia': 2,
 'NavelOrange': 3,
 'Sandsugaroranges': 4,
 'apple': 5,
 'banana': 6,
 'carrot': 7,
 'cherries': 8,
 'cucumber': 9,
 'durian': 10,
 'grape': 11,
 'hamimelon': 12,
 'kiwi': 13,
 'lemon': 14,
 'lichee': 15,
 'longan': 16,
 'mango': 17,
 'pear': 18,
 'pineapple': 19,
 'pitaya': 20,
 'pomegranate': 21,
 'strawberry': 22,
 'tomato': 23,
 'watermelon': 24}

In [20]:
# Mapping relationship: index number to category
idx_to_labels = {y:x for x,y in train_dataset.class_to_idx.items()}

In [21]:
idx_to_labels

{0: 'CherryTomatoes',
 1: 'Mangosteen',
 2: 'MomordicaCharantia',
 3: 'NavelOrange',
 4: 'Sandsugaroranges',
 5: 'apple',
 6: 'banana',
 7: 'carrot',
 8: 'cherries',
 9: 'cucumber',
 10: 'durian',
 11: 'grape',
 12: 'hamimelon',
 13: 'kiwi',
 14: 'lemon',
 15: 'lichee',
 16: 'longan',
 17: 'mango',
 18: 'pear',
 19: 'pineapple',
 20: 'pitaya',
 21: 'pomegranate',
 22: 'strawberry',
 23: 'tomato',
 24: 'watermelon'}

In [78]:
# Save as local npy file
np.save('idx_to_labels.npy', idx_to_labels)
np.save('labels_to_idx.npy', train_dataset.class_to_idx)

## Define the data loader DataLoader

In [22]:
from torch.utils.data import DataLoader

In [23]:
BATCH_SIZE = 32

# Data loader for the training set
train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=4
                         )

# Data Loader for Test Sets
test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=4
                        )

## View images and annotations for a batch

In [24]:
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler

## Choosing a Transfer Learning Training Approach

In [25]:
model = models.resnet18(pretrained=False) # Load only the model structure, not the pre-training weight parameters

model.fc = nn.Linear(model.fc.in_features, n_class)

optimizer = optim.Adam(model.parameters())

## Training configuration

In [26]:
model = model.to(device)

# Cross Entropy Loss Function
criterion = nn.CrossEntropyLoss() 

# Training rounds Epoch
EPOCHS = 30

# Learning rate reduction strategies
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

## Functions: training on a training set

In [27]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [28]:
def train_one_batch(images, labels):
    '''
    Run a batch of training and return the training log for the current batch.
    '''
    
    # Get a batch of data and annotations
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images) # Input model to perform forward prediction
    loss = criterion(outputs, labels) # Calculate the average cross-entropy loss function value for each sample in the current batch.
    
    # Optimising update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Get the label category and prediction category of the current batch
    _, preds = torch.max(outputs, 1) # Get the prediction categories for all images in the current batch
    preds = preds.cpu().numpy()
    loss = loss.detach().cpu().numpy()
    outputs = outputs.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    log_train = {}
    log_train['epoch'] = epoch
    log_train['batch'] = batch_idx
    # Calculation of disaggregated assessment indicators
    log_train['train_loss'] = loss
    log_train['train_accuracy'] = accuracy_score(labels, preds)
    # log_train['train_precision'] = precision_score(labels, preds, average='macro')
    # log_train['train_recall'] = recall_score(labels, preds, average='macro')
    # log_train['train_f1-score'] = f1_score(labels, preds, average='macro')
    
    return log_train

## Functions: Evaluate on the whole test set

In [29]:
def evaluate_testset():
    '''
    Evaluate on the entire test set and return a log of categorised evaluation metrics
    '''

    loss_list = []
    labels_list = []
    preds_list = []
    
    with torch.no_grad():
        for images, labels in test_loader: # Generate a batch of data and annotations
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images) # Input model to perform forward prediction

            # Get label categories and prediction categories for the entire test set
            _, preds = torch.max(outputs, 1) # Get the prediction categories for all images in the current batch
            preds = preds.cpu().numpy()
            loss = criterion(outputs, labels) # From logit, calculate the average cross-entropy loss function for each sample in the current batch.
            loss = loss.detach().cpu().numpy()
            outputs = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

            loss_list.append(loss)
            labels_list.extend(labels)
            preds_list.extend(preds)
        
    log_test = {}
    log_test['epoch'] = epoch
    
    # Calculation of disaggregated assessment indicators
    log_test['test_loss'] = np.mean(loss_list)
    log_test['test_accuracy'] = accuracy_score(labels_list, preds_list)
    log_test['test_precision'] = precision_score(labels_list, preds_list, average='macro')
    log_test['test_recall'] = recall_score(labels_list, preds_list, average='macro')
    log_test['test_f1-score'] = f1_score(labels_list, preds_list, average='macro')
    
    return log_test

## Before training starts, keep a log ##

In [30]:
epoch = 0
batch_idx = 0
best_test_accuracy = 0

In [33]:
# Training log - training sets
df_train_log = pd.DataFrame()
log_train = {}
log_train['epoch'] = 0
log_train['batch'] = 0
images, labels = next(iter(train_loader))
log_train.update(train_one_batch(images, labels))
df_train_log = df_train_log._append(log_train, ignore_index=True)

In [34]:
df_train_log

,epoch,batch,train_loss,train_accuracy
0,0,0,3.3938172,0.03125


In [36]:
# Training log - test set
df_test_log = pd.DataFrame()
log_test = {}
log_test['epoch'] = 0
log_test.update(evaluate_testset())
df_test_log = df_test_log._append(log_test, ignore_index=True)

In [37]:
df_test_log

,epoch,test_loss,test_accuracy,test_precision,test_recall,test_f1-score
0,0.0,3.697069,0.046771,0.022265,0.046261,0.01971


## Create wandb visualisation project

In [43]:
import wandb

wandb.init(project='fruit25', name=time.strftime('%m%d%H%M%S'))

wandb: Currently logged in as: 13048152760 (ual_student_xintianyin). Use `wandb login --relogin` to force relogin


## Run the training

In [47]:
for epoch in range(1, EPOCHS+1):
    
    print(f'Epoch {epoch}/{EPOCHS}')
    
    ## training phase
    model.train()
    for images, labels in tqdm(train_loader): # Get a batch of data and annotations
        batch_idx += 1
        log_train = train_one_batch(images, labels)
        df_train_log = df_train_log._append(log_train, ignore_index=True)
        wandb.log(log_train)
        
    lr_scheduler.step()

    ## testing phase
    model.eval()
    log_test = evaluate_testset()
    df_test_log = df_test_log._append(log_test, ignore_index=True)
    wandb.log(log_test)
    
    # Save the latest best model files
    if log_test['test_accuracy'] > best_test_accuracy: 
        # Delete old best model files (if any)
        old_best_checkpoint_path = 'checkpoint/best-{:.3f}.pth'.format(best_test_accuracy)
        if os.path.exists(old_best_checkpoint_path):
            os.remove(old_best_checkpoint_path)
        # Save the new best model file
        best_test_accuracy = log_test['test_accuracy']
        new_best_checkpoint_path = 'checkpoint/best-{:.3f}.pth'.format(log_test['test_accuracy'])
        torch.save(model, new_best_checkpoint_path)
        print('Save the new best model', 'checkpoint/best-{:.3f}.pth'.format(best_test_accuracy))
        # best_test_accuracy = log_test['test_accuracy']

df_train_log.to_csv('TraininglogTrainingSets.csv', index=False)
df_test_log.to_csv('TrainingLogTestSet.csv', index=False)

Epoch 1/30


100%|██████████| 115/115 [00:05<00:00, 21.28it/s]


Epoch 2/30


100%|██████████| 115/115 [00:05<00:00, 21.56it/s]


Epoch 3/30


...

100%|██████████| 115/115 [00:05<00:00, 21.46it/s]


Epoch 29/30


100%|██████████| 115/115 [00:05<00:00, 21.00it/s]


Epoch 30/30


100%|██████████| 115/115 [00:05<00:00, 21.69it/s]


##  Evaluation on the test set

In [45]:
# Load the best model as the current model
model = torch.load('checkpoint/best-{:.3f}.pth'.format(best_test_accuracy))

## Evaluate on a test set

In [46]:
model.eval()
print(evaluate_testset())

{'epoch': 30, 'test_loss': 1.0977473, 'test_accuracy': 0.6648106904231625, 'test_precision': 0.6735762147736243, 'test_recall': 0.66314819409647, 'test_f1-score': 0.6627346390295513}
